# Notebook preparatorio

## Instalación de las librerías necesarias

In [3]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install joblib

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import joblib



## Carga de datos

Procedemos con la carga de datos de entrenamiento para el modelo de clasificación.

In [5]:
diabetes_data = pd.read_csv("./diabetes_data_train.csv")


## Preprocesamiento

Vamos a declarar dos funciones. La primera para hacer codificación Dummy (One Hot pero con N-1) de algunas variables categóricas y la segunda para reducir el número de valores de diagnóstico.

In [6]:

def dummy_encoding(col, dataframe):
    enc_df = pd.get_dummies(dataframe[col], columns=dataframe[col].unique(), drop_first=True, dtype=int, prefix=f"{col}_")
    dataframe = dataframe.join(enc_df)
    return dataframe, enc_df.columns


def diag_cluster(col, dataframe):
    diag_list = []
    for x in dataframe[col]:
        # Si la columna contiene 'V' o 'E' entonces se le asigna el valor 18
        if 'V' in x or 'E' in x: 
            diag_list.append(18)
            continue
        # En todos los demás casos, se le asigna el valor indicado según el rango.
        elif 1 <= float(x) <= 139:
            diag_list.append(1)
        elif 140 <= float(x) <= 239:
            diag_list.append(2)
        elif 240 <= float(x) <= 279:
            diag_list.append(3)
        elif 280 <= float(x) <= 289:
            diag_list.append(4)
        elif 290 <= float(x) <= 319:
            diag_list.append(5)
        elif 320 <= float(x) <= 389:
            diag_list.append(6)
        elif 390 <= float(x) <= 459:
            diag_list.append(7)
        elif 460 <= float(x) <= 519:
            diag_list.append(8)
        elif 520 <= float(x) <= 579:
            diag_list.append(9)
        elif 580 <= float(x) <= 629:
            diag_list.append(10)
        elif 630 <= float(x) <= 679:
            diag_list.append(11)
        elif 680 <= float(x) <= 709:
            diag_list.append(12)
        elif 710 <= float(x) <= 739:
            diag_list.append(13)
        elif 740 <= float(x) <= 759:
            diag_list.append(14)
        elif 760 <= float(x) <= 779:
            diag_list.append(15)
        elif 780 <= float(x) <= 799:
            diag_list.append(16)
        elif 800 <= float(x) <= 999:
            diag_list.append(17)
    dataframe[col] = diag_list

Invocamos las funciones para realizar la clusterización de las columnas con los 3 diagnósticos y luego la codificación dummy de las columnas de raza y los tres diagnósticos.

In [7]:
diag_cluster('diag_1',diabetes_data)
diag_cluster('diag_2',diabetes_data)
diag_cluster('diag_3',diabetes_data)

diabetes_data, cols_race = dummy_encoding('race', diabetes_data)
diabetes_data, cols_diag_1 = dummy_encoding('diag_1', diabetes_data)
diabetes_data, cols_diag_2 = dummy_encoding('diag_2', diabetes_data)
diabetes_data, cols_diag_3 = dummy_encoding('diag_3', diabetes_data)

extra_cols = list(cols_race)+list(cols_diag_1)+list(cols_diag_2)+list(cols_diag_3)


Finalmente seleccionamos las columnas finales del modelo

In [8]:
X=diabetes_data[['num_lab_procedures', 'num_medications', 'age', 'gender', 'time_in_hospital', 'number_diagnoses', 'num_procedures',
       'admission_source_id', 'admission_type_id',
       'discharge_disposition_id', 'insulin', 'diabetesMed', 'change',
       'metformin', 'glipizide', 'number_outpatient', 'glyburide',
       'rosiglitazone', 'pioglitazone', 'number_inpatient', 'glimepiride',
       'number_emergency']+extra_cols]
y=diabetes_data['readmitted']

Entrenamos un arbol de clasificación con ciertos hiperparámetros

In [9]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=20, min_samples_split=2, min_samples_leaf=3, random_state = 2)
clf.fit(X, y)

DecisionTreeClassifier(max_depth=20, min_samples_leaf=3, random_state=2)

Serializamos el modelo

In [10]:
joblib.dump(clf, 'clasificador_base.joblib')


['clasificador_base.joblib']